# Building AI Applications with ChatGPT

Sumudu Tennakoon, PhD
<hr>

# Build Chatbot for Q&A from Documents

In this notebook we will explore some basic fetures on Python programing language for those who have a prior programing expereince.

To learn more about Python, refeer to the following websites

- Python : https://www.python.org

To learn more about the Python packages we explore in this notebook, refer to the following websites

- OpenAI API : https://platform.openai.com/docs/api-reference
- LangChain : https://python.langchain.com/docs/get_started/introduction.html
- Steamlit : https://streamlit.io/
- FLask : https://flask.palletsprojects.com/en/2.3.x


### Python Library Installation

* Run below code cell to install required libraries before you continue. Ignore that if you already installed them.

In [ ]:
!pip install openai langchain sentence_transformers chromadb streamlit

### Load OpenAI API Key

In [33]:
import configparser
config = configparser.ConfigParser()
config.read(r'../../../config.ini') #Change to your path or assign API Key to openai_api_key (not recomended for production)

openai_api_key = config['SECRETS']['openai_api_key']

## Read Document

In [78]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"..\..\..\Data\01-3.1.3.DL.Implications of responsible AI - Practical guide unit 2019 09 17.pdf")
pages = loader.load_and_split()

In [79]:
print(pages[0].page_content)

Abstract  
AI has transformed from vision to reality, creating tangible benefits for people and enterprises around the 
world. But like any technology, it poses the risk of negative consequences when used improperly or 
irresponsibly. We want to share what we’re learni ng in our own journey in hopes that it can provide a useful 
perspective for other organizations navigating similar challenges.  
Learn about Microsoft’s perspective on the importance of engaging with AI in a responsible manner . 
The transformative potential of AI  
AI is the defining technology of our time. It is already enabling faster and more profound progress in nearly 
every field of human endeavor and helping to address some of society’s most daunting challenges —like 
providing remo te students with access to education and helping farmers produce enough food for our 
growing global population.  
At Microsoft, we believe that the computational intelligence of AI should be used to amplify the innate 
creativity an

## Split Text From Document Pages Into Smaller Chunks

In [80]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, 
                                               chunk_overlap=10,
                                               length_function = len,
                                               separators=['\n\n', '\n', '.', ' ', ''],
                                               add_start_index = True,
                                               )

text_chunks = text_splitter.split_documents(pages)

text_chunks

[Document(page_content='Abstract  \nAI has transformed from vision to reality, creating tangible benefits for people and enterprises around the \nworld. But like any technology, it poses the risk of negative consequences when used improperly or \nirresponsibly. We want to share what we’re learni ng in our own journey in hopes that it can provide a useful \nperspective for other organizations navigating similar challenges.  \nLearn about Microsoft’s perspective on the importance of engaging with AI in a responsible manner .', metadata={'source': '..\\..\\..\\Data\\01-3.1.3.DL.Implications of responsible AI - Practical guide unit 2019 09 17.pdf', 'page': 0, 'start_index': 0}),
 Document(page_content='The transformative potential of AI  \nAI is the defining technology of our time. It is already enabling faster and more profound progress in nearly \nevery field of human endeavor and helping to address some of society’s most daunting challenges —like \nproviding remo te students with access

## Create Vector Database and Retriver

In [81]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': False}
)

retriever = Chroma.from_documents(text_chunks, embeddings)

In [82]:
# query_result = embeddings.embed_query(query)

#doc_result = embeddings.embed_documents(docs)

## Search Documents

In [83]:
query = "what are the three consideration when deploying AI responsibly?"

relevant_documents = retriever.max_marginal_relevance_search(query, k=3)
relevant_documents

Number of requested results 20 is greater than number of elements in index 17, updating n_results = 17


[Document(page_content='Applying these ideas in your organization  \nThe following three questio ns can help you start to consider the ways your organization can develop and \ndeploy AI in a responsible manner.  \n1. How can you use a human -led approach to drive value for your business?  \n2. How will your organization’s foundational values affect your approach t o AI?  \n3. How will you monitor AI systems to ensure they are evolving responsibly?  \n \nComing up', metadata={'source': '..\\..\\..\\Data\\01-3.1.3.DL.Implications of responsible AI - Practical guide unit 2019 09 17.pdf', 'page': 2, 'start_index': 0}),
 Document(page_content='introduced into either training data or machine learning models. At Microsoft, our researchers are exploring \ntools and techniques for detecting and reducing bias within AI systems. Explore more about this and more in \nthe summary and resources unit of this module.  \nSensitive use cases  \nAnother illustration of our responsibility to mitigate unin

## Create LLM Object

In [100]:
from langchain.chat_models import ChatOpenAI

MODEL = "gpt-3.5-turbo"

chat_llm = ChatOpenAI(model=MODEL, 
                      temperature=0, 
                      max_tokens=200, 
                      openai_api_key=openai_api_key,
                      verbose=True)

## Create Prompt

In [101]:
from langchain.chains import RetrievalQA

# chain_type = {"stuff", "map_reduce", "refine"} 
# https://python.langchain.com/docs/modules/chains/additional/question_answering.html

qa = RetrievalQA.from_chain_type(llm=chat_llm, 
                                 chain_type="stuff", 
                                 retriever=retriever.as_retriever(k=3))


In [54]:
from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end. \
If you don't know the answer, just say that you don't know, don't try to make up an answer.\

{context}

Question: {question}
Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm=chat_llm, 
                                 chain_type="stuff", 
                                 retriever=retriever.as_retriever(enable_limit=True,k=3), 
                                 chain_type_kwargs=chain_type_kwargs, 
                                 return_source_documents=True)




In [52]:
query = "Can you name a sensitive technology?"
result = qa({"query": query})
result

{'query': 'Can you name a sensitive technology?',
 'result': 'Facial recognition technology',
 'source_documents': [Document(page_content='the technology we release into the world through deliberate planning and continual oversight.  \nNovel threats  \nWe were reminded of this responsibility in 2016 w hen we released a chatbot on Twitter called Tay. We \ntaught Tay to learn unsupervised from interactions with Twitter users, so she could better replicate human \ncommunication and personality traits. However, within 24 hours users realized that she could lear n and', metadata={'source': '..\\..\\..\\Data\\01-3.1.3.DL.Implications of responsible AI - Practical guide unit 2019 09 17.pdf', 'page': 0, 'start_index': 2304}),
  Document(page_content='like facial recognition. Recently, there has been a growing demand for facial recognition technology, \nespecially from law enforcement organizations that see the potential of the technology for use cases like \nfinding missing children. However, 

## Add new Document

In [85]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"..\..\..\Data\01-3.1.4.DL.Responsible AI - Establishing guiding principles unit 2019 09 17.pdf")
pages = loader.load_and_split()

In [86]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, 
                                               chunk_overlap=10,
                                               length_function = len,
                                               separators=['\n\n', '\n', '.', ' ', ''],
                                               add_start_index = True,
                                               )

text_chunks = text_splitter.split_documents(pages)

text_chunks

[Document(page_content='Abstract  \nIn the last unit, we discussed the societal implications of AI and the responsibility of businesses, \ngovernments, NGOs, and academic researchers to anticipate and mitigate unintended consequences of AI \ntechnology. In light of this responsibility, organizat ions are finding the need to create internal policies and \npractices to guide their AI efforts, whether they are deploying third -party AI solutions or developing their \nown.', metadata={'source': '..\\..\\..\\Data\\01-3.1.4.DL.Responsible AI - Establishing guiding principles unit 2019 09 17.pdf', 'page': 0, 'start_index': 0}),
 Document(page_content='own.  \nAt Microsoft, we’ve recognized six principles that we believe should guide AI development and use: fairness, \nreliability and safety, privacy and security, inclusiveness, transparency, and accountability. For us, these \nprinciples are the cornerstone of a responsible and trustworthy approach to AI, especially as intelligent \ntechnolog

In [87]:
retriever.add_documents(text_chunks)

['d799296e-2c3c-11ee-9e57-64d69a653a55',
 'd799296f-2c3c-11ee-830f-64d69a653a55',
 'd7992970-2c3c-11ee-8895-64d69a653a55',
 'd7992971-2c3c-11ee-a2e5-64d69a653a55',
 'd7992972-2c3c-11ee-b766-64d69a653a55',
 'd7992973-2c3c-11ee-9096-64d69a653a55',
 'd7992974-2c3c-11ee-8c7c-64d69a653a55',
 'd7992975-2c3c-11ee-afeb-64d69a653a55',
 'd7992976-2c3c-11ee-9b34-64d69a653a55',
 'd7992977-2c3c-11ee-87a4-64d69a653a55',
 'd7992978-2c3c-11ee-81f9-64d69a653a55',
 'd7992979-2c3c-11ee-a328-64d69a653a55',
 'd799297a-2c3c-11ee-a8c7-64d69a653a55',
 'd799297b-2c3c-11ee-868c-64d69a653a55',
 'd799297c-2c3c-11ee-9d61-64d69a653a55',
 'd799297d-2c3c-11ee-b5bf-64d69a653a55',
 'd799297e-2c3c-11ee-9a22-64d69a653a55',
 'd799297f-2c3c-11ee-a4a1-64d69a653a55',
 'd7992980-2c3c-11ee-b6c5-64d69a653a55',
 'd7992981-2c3c-11ee-a3c7-64d69a653a55']

In [88]:
# from langchain.vectorstores import Chroma
# from langchain.embeddings import HuggingFaceEmbeddings

# embeddings = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/all-mpnet-base-v2",
#     model_kwargs={'device': 'cuda'},
#     encode_kwargs={'normalize_embeddings': False}
# )

# retriever = Chroma.from_documents(text_chunks, embeddings)

In [95]:
from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end. \
If you don't know the answer, just say that you don't know, don't try to make up an answer.\
If answer contain a list, output as a bulleted or numbered list.
{context}

Question: {question}
Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm=chat_llm, 
                                 chain_type="stuff", 
                                 retriever=retriever.as_retriever(enable_limit=True,k=3), 
                                 chain_type_kwargs=chain_type_kwargs, 
                                 return_source_documents=True)




In [96]:
query = "what are the Microsoft’s six guiding principles?"
result = qa({"query": query})
result

{'query': 'what are the Microsoft’s six guiding principles?',
 'result': '1. Fairness\n2. Reliability and safety\n3. Privacy and security\n4. Inclusiveness\n5. Transparency\n6. Accountability',
 'source_documents': [Document(page_content='own.  \nAt Microsoft, we’ve recognized six principles that we believe should guide AI development and use: fairness, \nreliability and safety, privacy and security, inclusiveness, transparency, and accountability. For us, these \nprinciples are the cornerstone of a responsible and trustworthy approach to AI, especially as intelligent \ntechnology  becomes more prevalent in the products and services we use every day.  \nMicros oft’s six guiding  principles   \n    Fairness', metadata={'source': '..\\..\\..\\Data\\01-3.1.4.DL.Responsible AI - Establishing guiding principles unit 2019 09 17.pdf', 'page': 0, 'start_index': 440}),
  Document(page_content='In the next two units, we have outlined some of the steps Microsoft and TD Bank are taking to prioriti

In [97]:
print(result['result'])

1. Fairness
2. Reliability and safety
3. Privacy and security
4. Inclusiveness
5. Transparency
6. Accountability


In [98]:
query = "why we need responsible approach to AI?"
result = qa({"query": query})
result

{'query': 'why we need responsible approach to AI?',
 'result': 'To mitigate the risk of negative consequences when AI is used improperly or irresponsibly.',
 'source_documents': [Document(page_content='Applying these ideas in your organization  \nThe following three questio ns can help you start to consider the ways your organization can develop and \ndeploy AI in a responsible manner.  \n1. How can you use a human -led approach to drive value for your business?  \n2. How will your organization’s foundational values affect your approach t o AI?  \n3. How will you monitor AI systems to ensure they are evolving responsibly?  \n \nComing up', metadata={'source': '..\\..\\..\\Data\\01-3.1.3.DL.Implications of responsible AI - Practical guide unit 2019 09 17.pdf', 'page': 2, 'start_index': 0}),
  Document(page_content='Abstract  \nAI has transformed from vision to reality, creating tangible benefits for people and enterprises around the \nworld. But like any technology, it poses the risk o

In [99]:
print(result['result'])

To mitigate the risk of negative consequences when AI is used improperly or irresponsibly.


<hr/>
Last update 2023-07-24 by Sumudu Tennakoon

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.